In [14]:
from datasets import load_dataset, concatenate_datasets
import pandas as pd
from sklearn.utils import resample
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
from optimum.bettertransformer import BetterTransformer
import torch

In [15]:
print("Torch version:",torch.__version__)

print("Is CUDA enabled?",torch.cuda.is_available())

Torch version: 2.1.1
Is CUDA enabled? True


In [16]:
dataset = load_dataset("financial_phrasebank", 'sentences_50agree')

Found cached dataset financial_phrasebank (C:/Users/Gusso/.cache/huggingface/datasets/financial_phrasebank/sentences_50agree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141)
100%|██████████| 1/1 [00:00<00:00, 1000.07it/s]


In [17]:
neutrals = dataset.filter(lambda example: example["label"] == 1)
positives = dataset.filter(lambda example: example["label"] == 2)
negatives = dataset.filter(lambda example: example["label"] == 0)

short_positives = positives['train'].select(range(len(negatives['train'])))

print(short_positives)

downsized_dataset = concatenate_datasets([short_positives, negatives['train']])

print(downsized_dataset)

Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-4069f8bdc2072a81.arrow
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-255302c37e6b94c9.arrow
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-913aca77f5bc0114.arrow


Dataset({
    features: ['sentence', 'label'],
    num_rows: 604
})
Dataset({
    features: ['sentence', 'label'],
    num_rows: 1208
})


In [18]:
def replace_values(example):
    # Replace values in the label column
    if "label" in example:
        example["label"] = 1 if example["label"] == 2 else example["label"]
    return example

# Use the map function to apply the custom mapping
downsized_dataset = downsized_dataset.map(replace_values)
downsized_dataset = downsized_dataset.train_test_split(test_size=0.1, shuffle=True)
print(downsized_dataset)


Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-9ab259d81c7af6d1.arrow


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 1087
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 121
    })
})


In [19]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding='max_length', truncation=True)

tokenized_datasets = downsized_dataset['train'].map(tokenize_function, batched=True)
tokenized_datasets_eval = downsized_dataset['test'].map(tokenize_function, batched=True)


In [20]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

model = model.to(0)



metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_eval_batch_size=16, per_device_train_batch_size=16)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets_eval,
    compute_metrics=compute_metrics,
)

In [22]:
trainer.train()

                                                 
 34%|███▎      | 137/408 [00:18<01:30,  2.98it/s]

{'eval_loss': 0.17680807411670685, 'eval_accuracy': 0.9669421487603306, 'eval_runtime': 0.696, 'eval_samples_per_second': 173.85, 'eval_steps_per_second': 22.988, 'epoch': 1.0}


                                                 
 67%|██████▋   | 273/408 [00:38<00:48,  2.77it/s]

{'eval_loss': 0.18295447528362274, 'eval_accuracy': 0.9669421487603306, 'eval_runtime': 0.7715, 'eval_samples_per_second': 156.835, 'eval_steps_per_second': 20.739, 'epoch': 2.0}


                                                 
100%|██████████| 408/408 [00:54<00:00,  7.44it/s]

{'eval_loss': 0.1199985221028328, 'eval_accuracy': 0.9752066115702479, 'eval_runtime': 0.635, 'eval_samples_per_second': 190.55, 'eval_steps_per_second': 25.197, 'epoch': 3.0}
{'train_runtime': 54.843, 'train_samples_per_second': 59.461, 'train_steps_per_second': 7.439, 'train_loss': 0.15642192316990272, 'epoch': 3.0}


TrainOutput(global_step=408, training_loss=0.15642192316990272, metrics={'train_runtime': 54.843, 'train_samples_per_second': 59.461, 'train_steps_per_second': 7.439, 'train_loss': 0.15642192316990272, 'epoch': 3.0})

In [23]:
model = BetterTransformer.transform(model)

trainer2 = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets_eval,
    compute_metrics=compute_metrics,
)

The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


In [24]:
validation_dataset = load_dataset('zeroshot/twitter-financial-news-sentiment')

def filter_function(example):
    return example["label"] != 2

# Use the filter method to remove records
filtered_dataset = validation_dataset.filter(filter_function)

def tokenize_function2(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True)

tokenized_datasets_validation = filtered_dataset.map(tokenize_function2, batched=True)

# Print the modified dataset
print(dataset)

Found cached dataset csv (C:/Users/Gusso/.cache/huggingface/datasets/zeroshot___csv/zeroshot--twitter-financial-news-sentiment-ccca0f3c622c5b67/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 2/2 [00:00<00:00, 1001.74it/s]
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\zeroshot___csv\zeroshot--twitter-financial-news-sentiment-ccca0f3c622c5b67\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-b2d98c45395bc49c.arrow
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\zeroshot___csv\zeroshot--twitter-financial-news-sentiment-ccca0f3c622c5b67\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-98a6e9e55fa382e5.arrow
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\zeroshot___csv\zeroshot--twitter-financial-news-sentiment-ccca0f3c622c5b67\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cac

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 4846
    })
})


In [25]:
trainer2.evaluate(eval_dataset=tokenized_datasets_validation['validation'])

100%|██████████| 103/103 [00:01<00:00, 53.42it/s]


{'eval_loss': 0.9385751485824585,
 'eval_accuracy': 0.8004866180048662,
 'eval_runtime': 1.962,
 'eval_samples_per_second': 418.96,
 'eval_steps_per_second': 52.497}